# Merge

One of the most powerful features of pandas is the ability to merge datasets together. This is essential for data journalism, where you often need to combine information from multiple sources to tell a complete story.

In our helicopter accident analysis, we want to combine accident data with flight hour data to calculate accident rates - a much more meaningful metric than raw accident counts.

## Setup: Load both datasets

Let's start by loading both our accident data and the FAA survey data that contains flight hours:

In [ ]:
import pandas as pd

# Load accident data
accident_list = pd.read_csv("https://raw.githubusercontent.com/palewire/first-python-notebook/main/docs/src/_static/ntsb-accidents.csv")
accident_list["latimes_make_and_model"] = accident_list["latimes_make_and_model"].str.upper()

print(f"Loaded {len(accident_list)} accidents")
accident_list.head()

In [ ]:
# Load FAA survey data with flight hours
survey = pd.read_csv("https://raw.githubusercontent.com/palewire/first-python-notebook/main/docs/src/_static/faa-survey.csv")
survey["latimes_make_and_model"] = survey["latimes_make_and_model"].str.upper()

print(f"Loaded {len(survey)} helicopter models with flight hour data")
survey.head()

## Prepare data for merging

Before merging, we need to create a summary of accidents by helicopter model:

In [ ]:
# Count accidents by make and model
accident_counts = accident_list.groupby(["latimes_make", "latimes_make_and_model"]).size().rename("accidents").reset_index()

print(f"Accident counts for {len(accident_counts)} helicopter models")
accident_counts.head()

## Basic merge operation

Now we can merge the accident counts with the flight hour data. We'll use the `merge` function to combine the datasets on the `latimes_make_and_model` column:

In [ ]:
# Merge the datasets
merged_list = pd.merge(accident_counts, survey, on="latimes_make_and_model")

print(f"Merged dataset has {len(merged_list)} helicopter models")
merged_list.head()

## Understanding the merge

Let's examine what happened during the merge:

In [ ]:
print(f"Original accident counts: {len(accident_counts)} models")
print(f"Original survey data: {len(survey)} models")
print(f"Merged result: {len(merged_list)} models")
print("\nThe merge kept only models that appear in both datasets (inner join)")

## Calculate accident rates

Now that we have both accident counts and flight hours in the same dataset, we can calculate meaningful accident rates:

In [ ]:
# Calculate accident rate per flight hour
merged_list["per_hour"] = merged_list["accidents"] / merged_list["total_hours"]

# Calculate accident rate per 100,000 flight hours (more readable)
merged_list["per_100k_hours"] = (merged_list["accidents"] / merged_list["total_hours"]) * 100_000

print("Added accident rate calculations:")
merged_list[["latimes_make_and_model", "accidents", "total_hours", "per_100k_hours"]].head()

## Analyze the results

Now we can see which helicopter models have the highest accident rates:

In [ ]:
# Sort by accident rate to see the most dangerous models
dangerous_models = merged_list.sort_values("per_100k_hours", ascending=False)
print("Top 10 helicopter models by accident rate (per 100k flight hours):")
dangerous_models[["latimes_make_and_model", "accidents", "total_hours", "per_100k_hours"]].head(10)

## Different types of merges

Pandas supports different types of merges. Let's explore what happens with different join types:

In [ ]:
# Left join - keep all accident data, even without flight hours
left_merge = pd.merge(accident_counts, survey, on="latimes_make_and_model", how="left")
print(f"Left merge result: {len(left_merge)} models (includes all accident data)")

# Check how many models don't have flight hour data
missing_hours = left_merge["total_hours"].isnull().sum()
print(f"Models with accidents but no flight hour data: {missing_hours}")

In [ ]:
# Right join - keep all survey data, even without accidents
right_merge = pd.merge(accident_counts, survey, on="latimes_make_and_model", how="right")
print(f"Right merge result: {len(right_merge)} models (includes all survey data)")

# Fill missing accident counts with 0
right_merge["accidents"] = right_merge["accidents"].fillna(0)
no_accidents = (right_merge["accidents"] == 0).sum()
print(f"Models with flight hours but no recorded accidents: {no_accidents}")

## Key insights from the merge

The merge operation revealed important insights about helicopter safety:

In [ ]:
# Summary statistics
print("Summary of accident rates:")
print(f"Average accident rate: {merged_list['per_100k_hours'].mean():.2f} per 100k hours")
print(f"Median accident rate: {merged_list['per_100k_hours'].median():.2f} per 100k hours")
print(f"Highest accident rate: {merged_list['per_100k_hours'].max():.2f} per 100k hours")
print(f"Lowest accident rate: {merged_list['per_100k_hours'].min():.2f} per 100k hours")

Merging datasets is a fundamental skill in data journalism. It allows you to combine information from different sources to create more meaningful analyses, like calculating rates instead of just raw counts. This helicopter accident analysis demonstrates how merging accident data with flight hour data provides much more insight than looking at accidents alone.